In [2]:
from vatic.engines import Simulator
from vatic.data.loaders import load_input, RtsLoader
from vatic.engines import Simulator

import pandas as pd
import numpy as np

from pathlib import Path
import dill as pickle
import os
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
RUC_MIPGAPS = {'Texas-7k': 0.01}
SCED_HORIZONS = {'Texas-7k': 4}

grid = 'Texas-7k'  
num_days = 1  
start_date = '2018-01-19' #for Texas pick a date in 2018
init_state_file = None 
template, gen_data, load_data = load_input(grid, start_date, 
    num_days=num_days, init_state_file=init_state_file)

In [ ]:
siml = Simulator(template, gen_data, load_data, None,
                pd.to_datetime(start_date).date(), 1, solver='gurobi',
                solver_options={"Threads": 8}, run_lmps=False, mipgap=RUC_MIPGAPS[grid],
                load_shed_penalty = 1e4, reserve_shortfall_penalty = 1e3,
                reserve_factor=0.05, output_detail=3,
                prescient_sced_forecasts=True, ruc_prescience_hour=0,
                ruc_execution_hour=16, ruc_every_hours=24,
                ruc_horizon=48, sced_horizon=SCED_HORIZONS[grid],
                lmp_shortfall_costs=False,
                enforce_sced_shutdown_ramprate=False,
                no_startup_shutdown_curves=False,
                init_ruc_file=None, verbosity=0,
                output_max_decimals=4, create_plots=False,
                renew_costs=None, save_to_csv=False, 
                last_conditions_file=None,)
report_dfs = siml.simulate()

## Part A

**(i)**

Run a series of simulations by scaling renewable generation using the following factors:

$$
\alpha = 1.0, 0.75, 0.5, 0.25, 0.0.
$$

That is, multiply all renewable generation (wind and solar) by each of these scaling parameters and rerun the `Simulator` for each case.  
Then, **plot the results** (for example, total system cost, market price and thermal dispatch) against $\alpha$.

Finally, **comment on the observed results**, discuss how reducing renewable generation impacts total cost and prices.  

-----

**(ii)**

Examine whether there is a **linear relationship** between renewable generation and the key output variables (e.g., total cost, prices, and thermal dispatch).

In particular, does the change in each output appear **proportional** to the change in $\alpha$, or do you observe **nonlinear effects** (such as thresholds, curvatures, or sharp transitions in the plots)?

This analysis can be done by taking the **average value throughout the day** for each output variable and comparing these averages across different values of $\alpha$.  


## Part B

Wind generation is inherently uncertain due to the variability of weather systems. To evaluate how fluctuations influences system operation and market outcomes, we introduce controlled perturbations to the baseline wind production data. We consider two cases. (Here wind turbines refer to all sources that have 'Wind' in their name)

**(i) Independent Forecast Errors**

In the first case, we assume that the variations for each wind turbine are independent across generators. That is, each turbine’s production is adjusted by an error term drawn independently from the same statistical distribution.

We want to find the answer to "what if wind were 15% higher on average that day? ($\mu = 15%$).
The volatility around this average bias is $\sigma = 20%$.

Each turbine’s error is drawn independently of the others, $\epsilon_i(t) \sim \mathcal{N}(\mu, \sigma^2)$.

Mathematically, for turbine i at time t: $ production_i(t) = baseline_i(t) \cdot (1 + \epsilon_i(t))$

------

**(ii) Correlated Errors (Shared Weather)**

Turbines experience the correlated/similar regional weather, so we model a shared shock plus a small local wiggle (volatility):
$$
\epsilon_{i}(t) \;=\; \mu \;+\; \sigma\!\left(\sqrt{\rho}\,z^{\text{common}}(t) \;+\; \sqrt{1-\rho}\,z^{\text{local}}_{i}(t)\right),
$$
with $z^{\text{common}}(t),z^{\text{local}}_{i}(t) \stackrel{\text{i.i.d.}}{\sim} \mathcal{N}(0,1)$, correlation $\rho=0.8$, and the same $\mu=0.15$, $\sigma=0.20$ as above. Properties:

* $z^{\text{common}}(t)$ is the same across all sources at a given time $t$.
* At any hour $t$, turbines move mostly together (correlation $\rho$ across units).
* Across time, errors remain independent (hourly weather surprises are independent draws).